In [3]:
import glob
import pandas as pd
import numpy as np

In [2]:
path="Dataset/ELAB_Educ_data"
files = glob.glob(path + "/*.xlsx")

# 1 Read EDUC Data

In [3]:
# data_frame = pd.DataFrame()
# content = []
# for filename in files:
    
#     # reading content of csv file
#     # content.append(filename)
#     df = pd.read_excel(filename, index_col=None)
#     content.append(df)

# data_frame = pd.concat(content)

In [4]:
df=pd.read_excel("Dataset/ELAB_Educ_data/DC_1819_final_January_28th_22.xlsx")
print(df.shape)
df.head()

(242, 39)


,state,year,distid_stateassigned,schoolid_stateassigned,distname,schoolname,ncesdistid_admin,ncesdistid_geo,census_id,ncesid,...,distexclu_raw_DC,schtype_raw_DC,ward_raw_DC,countenroll,noaudit,nositecen,adfinvars,distflex,flag_nerds,flag_f33
0,DC,2018-2019,178,233,Academy of Hope Adult PCS,Academy of Hope Adult PCS,NaN,NaN,NaN,NaN,...,721987,Charter,5,1,0,0,0,1,0,0
1,DC,2018-2019,155,217,Achievement Preparatory Academy PCS,Achievement Preparatory Academy PCS - Wahler P...,1100072.0,1100072.0,NaN,1.100072e+11,...,5331797,Charter,8,1,0,0,0,1,0,0
2,DC,2018-2019,155,1100,Achievement Preparatory Academy PCS,Achievement Preparatory Academy PCS - Wahler P...,1100072.0,1100072.0,NaN,1.100072e+11,...,5331797,Charter,8,1,0,0,0,1,0,0
3,DC,2018-2019,103,140,AppleTree Early Learning PCS,AppleTree Early Learning Center PCS - Columbia...,1100054.0,1100054.0,NaN,1.100054e+11,...,7363789,Charter,1,1,0,0,0,1,0,0
4,DC,2018-2019,103,3072,AppleTree Early Learning PCS,AppleTree Early Learning Center PCS - Douglas ...,1100054.0,1100054.0,NaN,1.100054e+11,...,7363789,Charter,8,1,0,0,0,1,0,0


## 1.1 Missing Value

In [5]:
# Find Missing value 
df.isna().sum()

state                          0
year                           0
distid_stateassigned           0
schoolid_stateassigned         0
distname                       0
schoolname                     0
ncesdistid_admin               8
ncesdistid_geo                 8
census_id                    126
ncesid                        10
ncesenroll                    21
gradespan                     12
level                         12
enroll_raw_DC                  0
enrollmetric                   0
pp_stloc_raw_DC                0
pp_fed_raw_DC                  0
pp_total_raw_DC                0
pp_site_stloc_raw_DC          43
pp_site_fed_raw_DC            43
pp_site_raw_DC                43
pp_centshare_stloc_raw_DC     43
pp_centshare_fed_raw_DC       43
pp_centshare_raw_DC           43
schoolstloc_raw_DC             0
schoolfed_raw_DC               0
schooltot_raw_DC               0
pp_total_norm_NERDS           47
disttotwexclu_raw_DC           0
distexclu_raw_DC               0
schtype_ra

In [6]:
df.gradespan

0        NaN
1      PK-03
2      04-08
3      PK-PK
4      PK-PK
       ...  
237    05-08
238    09-12
239    09-12
240    PK-05
241      NaN
Name: gradespan, Length: 242, dtype: object

### 1.1.1 IDs

Census_id is census bureau 14-digit government id, didn't contian valuable information. Can leave it to blank
ncesid is NCES school id，also can leave to blank.

### 1.1.2 ncesenroll
enrollment from NCES Common Core of Data called students* in the NCES downloadable files
Find a way to fill it with other variables

### 1.1.3 level and grade span
13 rows don't have level and grade span, most of these are PCS (Professional Children School), school for child actors Fill these cells

In [7]:
df.gradespan.unique()

array([nan, 'PK-03', '04-08', 'PK-PK', '05-12', 'PK-02', 'PK-05', '09-12',
       'PK-04', '05-08', 'PK-KG', 'PK-08', '08-08', '06-07', '06-12',
       '09-11', '06-08', '06-06', '03-12', '07-12', '01-05', 'PK-01',
       'KG-08', '04-07', 'KG-05', 'PK-06', '01-04', 'KG-04', '08-12',
       'PK-12', '04-05'], dtype=object)

In [8]:
#Column census_id can be dropped 

#level data
level_na = df[df.level.isna()].reset_index(drop=True)
level_rest=df[-df.level.isna()].reset_index(drop=True)
# Add a new level value "PCS"
level_na.level=np.where(level_na.schoolname.str.contains("PCS"),5,
        np.where(level_na.schoolname.str.contains("Elementary"),2,"")
        )
level_na.gradespan=np.where(level_na.schoolname.str.contains("PCS"),"PCS",
        np.where(level_na.schoolname.str.contains("Elementary"),"01-05","")
        )

pd.concat([level_na,level_rest]).reset_index(drop=True)

,state,year,distid_stateassigned,schoolid_stateassigned,distname,schoolname,ncesdistid_admin,ncesdistid_geo,census_id,ncesid,...,distexclu_raw_DC,schtype_raw_DC,ward_raw_DC,countenroll,noaudit,nositecen,adfinvars,distflex,flag_nerds,flag_f33
0,DC,2018-2019,178,233,Academy of Hope Adult PCS,Academy of Hope Adult PCS,NaN,NaN,NaN,NaN,...,721987,Charter,5,1,0,0,0,1,0,0
1,DC,2018-2019,119,126,Briya PCS,Briya PCS,NaN,NaN,NaN,NaN,...,700353,Charter,5,1,0,0,0,1,0,0
2,DC,2018-2019,162,1119,Carlos Rosario International PCS,Carlos Rosario International PCS,NaN,NaN,NaN,NaN,...,576499,Charter,1,1,0,0,0,1,0,0
3,DC,2018-2019,109,153,Cesar Chavez PCS for Public Policy,Cesar Chavez PCS for Public Policy - Capitol Hill,1100005.0,1100005.0,NaN,1.100005e+11,...,2638812,Charter,6,1,0,0,0,1,0,1
4,DC,2018-2019,109,127,Cesar Chavez PCS for Public Policy,Cesar Chavez PCS for Public Policy - Chavez Prep,1100005.0,1100005.0,NaN,1.100005e+11,...,2638812,Charter,1,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,DC,2018-2019,185,263,Washington Global PCS,Washington Global PCS,1100102.0,1100102.0,NaN,1.100102e+11,...,1457869,Charter,6,1,0,0,0,1,0,0
238,DC,2018-2019,151,125,Washington Latin PCS,Washington Latin PCS - Middle School,1100019.0,1100019.0,NaN,1.100019e+11,...,318733,Charter,4,1,0,0,0,1,0,0
239,DC,2018-2019,151,1118,Washington Latin PCS,Washington Latin PCS - Upper School,1100019.0,1100019.0,NaN,1.100019e+11,...,318733,Charter,4,1,0,0,0,1,0,0
240,DC,2018-2019,194,283,Washington Leadership Academy PCS,Washington Leadership Academy PCS,1100103.0,1100103.0,NaN,1.100103e+11,...,1885439,Charter,5,1,0,0,0,1,1,1


### 1.3 Financial Variables
pp_site_stloc_raw_DC          
pp_site_fed_raw_DC            
pp_site_raw_DC                
pp_centshare_stloc_raw_DC     
pp_centshare_fed_raw_DC       
pp_centshare_raw_DC         

These features are the main feature we are going to study with. These rows with missing value may have to be dropped 

# 2.  DCPS DATA

In [19]:
sat=pd.read_excel("Dataset/DCPS/School Year 2018-2019 SAT Scores.xlsx",header=1,skiprows=4)
print(sat.shape)
sat.head()

(21, 6)


,School Code,School Name,Number of Test-takers,Evidence-Based Reading and Writing Average,Math Average,Total Average
0,402,Benjamin Banneker HS,206,557.281555,553.689331,1110.970825
1,436,Ron Brown College Preparatory High School,57,429.122803,410.175446,839.298218
2,442,Columbia Heights EC (CHEC),371,444.447449,428.840973,873.288391
3,450,Anacostia HS,101,381.287140,374.059418,755.346558
4,452,Ballou HS,240,396.666656,381.833344,778.500000


In [20]:
ap=pd.read_excel("Dataset/DCPS/School Year 2018-2019 AP Scores.xlsx",header=1,skiprows=4)
print(ap.shape)
ap.head()

(19, 4)


,School Name,Number Students with an AP Score,"Number Students with at Least One AP Score of 3, 4, or 5","Percent Students Scoring 3, 4, or 5"
0,Anacostia HS,34,0,0.000000
1,Ballou HS,52,9,0.173077
2,Benjamin Banneker HS,243,135,0.555556
3,Cardozo EC,98,30,0.306122
4,Columbia Heights EC (CHEC),473,134,0.283298


In [21]:
grad=pd.read_excel("Dataset/DCPS/DCPS 2018 Graduation Data Overall Subgroup.xlsx",header=1,skiprows=4)
print(grad.shape)
grad.head()

(195, 5)


,School Code,School Name,Subgroup,# Grads,% Grad
0,402,Benjamin Banneker HS,All,103,1.0
1,402,Benjamin Banneker HS,Asian,n < 10,NaN
2,402,Benjamin Banneker HS,At-Risk,24,1.0
3,402,Benjamin Banneker HS,Black,76,1.0
4,402,Benjamin Banneker HS,EL,n < 10,NaN
